# Building ML-models with PCA dimensionality reduction

Preprocessing and feature extraction is performed after splitting the data (therefore, only the samples in the training data are available for these steps). 

1. Models are built as pipelines of different steps. 
    - All inlcude a standardization step.
    - Then a PCA step follows to extract features, but the different models keep a different number of the n first PCs.
    - The models are named after how many PCs they keep.
2. The models are evaluated using leave one out cross validation. 
    - Each model (which uses a given number of extracted PCs as features) is therefore trained and evaluated 8 times 
        - All of the steps (the training) in the model is performed on 7 of the samples to fit the model
            - Therefore, the maximum number of PCs kept is 7 (minimum of n_samples and n_features) 
        - The fitted model is then evaluated by prediciting the flux of the target reaction in the 8th sample.
    - The average of these performances is reported and used to compare the model with models keeping a different number of PCs as features.

In [1]:
# import libraries
import pandas as pd
import numpy as np
from IPython.display import display

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn import svm

In [2]:
# import self-made functions
%run /Users/mariekececilia/Documents/master_thesis_code/methods.ipynb

In [3]:
#set a random seed to make reproducible results (used in PCA calculation)
seed = 0

## Load, clean and prepare data

In [4]:
#load transcriptional and fluxomic data
gexp, flux = load_gerosa()

#clean the transcriptional data (remove duplicates)
gexp, groups = clean_gexp_g(gexp)

#define features and log-transform them
X = gexp
X = np.log(X)

#define targets and extract their fluxes
target_labels = ['R_PPC', 'R_AKGDH', 'R_ICDHyr']
targets = dict()
for target_label in target_labels:
    targets[target_label] = flux[target_label]

## Create models/pipelines

Choose learning algorithms:

In [5]:
algorithms = dict({
    'lr': linear_model.LinearRegression(),
    'ridge_a1': linear_model.Ridge(),
    'lasso_a1': linear_model.Lasso(),
    'svr': svm.SVR(), 
    })

Get the models/pipelines:

In [6]:
# get a list of models to evaluate, name each by the dimensions they keep
def get_models(max_n_components, model):
    models = dict()
    for i in range(1, max_n_components+1):
        steps = [
            ('standardize', StandardScaler()),
            ('pca', PCA(n_components=i, random_state = seed)), 
            ('model', model)]
        models[str(i)] = Pipeline(steps=steps)
    return models

In [7]:
# get the models to evaluate (available samples is one less due to LOOCV)
max_n_components = min(X.shape[0]-1, X.shape[1]) 

models = dict()

# create combinations of PCs kept and learning algorithms used
for name, algorithm in algorithms.items():
     models[name]= get_models(max_n_components, algorithm)

## Learning

In [8]:
# evaluate the models and store results in nested dicts: reaction -> learning algorithm -> scores/preds of the different n first PCs kept
scores = dict()
preds = dict()
for y_name, y in targets.items():
    model_scores = dict()
    model_preds = dict()
    for model_name, pipelines in models.items():
        #get scores
        model_scores[model_name] = compare_models(pipelines, X, y)
        #get predictions
        model_preds[model_name] = get_models_preds(pipelines, X, y)
    scores[y_name] = model_scores
    preds[y_name] = model_preds

### Save data to use in other notebooks

Scores and predictions are visualized in the Notebook 'compare_models', which also merges the results with the results from all other models built.

In [9]:
scores_pca = scores
preds_pca = preds
%store scores_pca
%store preds_pca

Stored 'scores_pca' (dict)
Stored 'preds_pca' (dict)


### Scores and predictions

In [10]:
for y in targets.keys():
    for algorithm in algorithms.keys():
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> summary of all cv split scores for each number of PCs kept:')
        df = scores[y][algorithm][1].sort_values(by = 'average')
        df.index.name = 'PCs kept'
        df.columns.name = 'Score'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all individual cv-split scores for each number of PCs kept:')
        df = scores[y][algorithm][0]
        df.columns.name = 'PCs kept'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all predictions for each number of PCs kept (+ actual values):')
        df = preds[y][algorithm]
        df.index.name = 'Test set'
        df.columns.name = 'PCs kept'
        display(df)


R_PPC	(target)
lr	(learning algorithm)
–> summary of all cv split scores for each number of PCs kept:


Score,average,std
PCs kept,,
7,0.297126,0.439747
6,0.297126,0.439747
5,0.310365,0.471167
4,0.314939,0.467331
2,0.358261,0.557075
3,0.371358,0.569436
1,0.824399,1.576637



R_PPC	(target)
lr	(learning algorithm)
–> all individual cv-split scores for each number of PCs kept:


PCs kept,1,2,3,4,5,6,7
Test set,,,,,,,
Acetate,0.027282,0.178716,0.180808,0.144477,0.143887,0.143348,0.143348
Fructose,0.484160,0.291229,0.291422,0.280306,0.277259,0.272840,0.272840
Galactose,4.973507,1.816175,1.864150,1.532830,1.538454,1.441175,1.441175
Glucose,0.077964,0.089787,0.104066,0.057776,0.065356,0.066814,0.066814
Glycerol,0.491493,0.110607,0.110428,0.111963,0.092215,0.091620,0.091620
Gluconate,0.207011,0.094304,0.125406,0.092627,0.071792,0.069753,0.069753
Pyruvate,0.222655,0.252963,0.253356,0.255478,0.249557,0.248495,0.248495
Succinate,0.111116,0.032302,0.041228,0.044055,0.044397,0.042965,0.042965



R_PPC	(target)
lr	(learning algorithm)
–> all predictions for each number of PCs kept (+ actual values):


...

Score,average,std
PCs kept,,
6,1.091856,1.800532
7,1.091891,1.800616
3,1.103454,1.897451
4,1.107072,1.913749
5,1.107148,1.863674
2,1.156868,2.057864
1,1.394295,2.161368



R_ICDHyr	(target)
svr	(learning algorithm)
–> all individual cv-split scores for each number of PCs kept:


PCs kept,1,2,3,4,5,6,7
Test set,,,,,,,
Acetate,0.425926,0.357285,0.350516,0.321279,0.370480,0.376885,0.376885
Fructose,0.319528,0.052121,0.097536,0.158103,0.204809,0.212399,0.212401
Galactose,6.937306,6.452645,5.989697,6.039125,5.901968,5.714800,5.715058
Glucose,0.121373,0.034484,0.159766,0.088752,0.091665,0.100310,0.100296
Glycerol,0.786603,0.222029,0.178172,0.179866,0.194272,0.196228,0.196233
Gluconate,1.854310,1.601872,1.505628,1.492379,1.504309,1.515920,1.515934
Pyruvate,0.663956,0.438789,0.448078,0.477175,0.515178,0.519759,0.519776
Succinate,0.045362,0.095722,0.098240,0.099901,0.074506,0.098551,0.098544



R_ICDHyr	(target)
svr	(learning algorithm)
–> all predictions for each number of PCs kept (+ actual values):


PCs kept,actual,1,2,3,4,5,6,7
Test set,,,,,,,,
Acetate,4.696417,2.696089,3.018456,3.050246,3.187557,2.956490,2.926409,2.926409
Fructose,4.566119,3.107117,4.328130,4.120756,3.844203,3.630937,3.596281,3.596272
Galactose,0.496177,3.938312,3.697834,3.468130,3.492655,3.424601,3.331732,3.331860
Glucose,2.977971,3.339415,3.080663,2.502192,2.713670,2.704997,2.679251,2.679292
Glycerol,2.464836,4.403684,3.012102,2.904001,2.908177,2.943686,2.948506,2.948518
Gluconate,1.154034,3.293969,3.002647,2.891579,2.876290,2.890057,2.903457,2.903472
Pyruvate,7.979320,2.681401,4.478079,4.403962,4.171791,3.868552,3.831999,3.831859
Succinate,3.038949,3.176801,2.748054,2.740403,2.735357,2.812529,2.739458,2.739479
